In [ ]:
from sklearn.cluster import MiniBatchKMeans
import rasterio as rio
import numpy as np

def compute_cluster(img,k=4,random_seed=42):
    img_float = img.astype(np.float32)/10000
    samples = reshape_as_image(img_float).reshape(-1,img_float.shape[0])
    kmeans_pred = MiniBatchKMeans(n_clusters=k+1, random_state=random_seed,max_iter=10,batch_size=10000,reassignment_ratio=0).fit(samples)
    kmeans_pred_img = kmeans_pred.labels_.reshape(rws_rgb_img.shape[1], rws_rgb_img.shape[2]).astype(rio.uint16)
    return kmeans_pred_img